In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
path = 'PATH_TO_DATASET_IN_DRIVE'
df = pd.read_csv(path+"unified_dataset_ibge.csv", sep=',')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (45,46,48,76,93,109,112,125,135,177,179,186,192,205,212,220,237) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()

,mc_cri_aleitamento,mc_cri_analise_conclusao,mc_cri_apgar_1_minuto,mc_cri_apgar_5_minuto,mc_cri_comentario_encaminhamento,mc_cri_data_certidao_nascimento,mc_cri_data_obito,mc_cri_data_teste_orelhinha,mc_cri_data_teste_pezinho,mc_cri_data_triagem,...,mc_ips_id,mc_ins_id,mc_mul_id_y,mc_dae_mun_atend_id,mc_dae_ptr_id,mc_dae_pma_id,mc_dae_ten_id,mc_dae_uns_id,ativo,dataexclusao_y
0,0.0,NaN,9.0,10.0,NaN,2014-10-05,NaN,NaN,NaN,NaN,...,28491,1938078,39798,113.0,NaN,1.0,1.0,NaN,True,NaN
1,NaN,NaN,NaN,NaN,NaN,2010-08-19,NaN,NaN,NaN,NaN,...,28491,1938078,39798,113.0,NaN,1.0,1.0,NaN,True,NaN
2,1.0,NaN,3.0,9.0,NaN,2009-08-18,NaN,NaN,NaN,NaN,...,30490,5319596,42180,114.0,5.0,1.0,7.0,NaN,True,NaN
3,1.0,NaN,6.0,9.0,NaN,2009-08-18,NaN,NaN,NaN,NaN,...,30490,5319596,42180,114.0,5.0,1.0,7.0,NaN,True,NaN
4,0.0,NaN,9.0,10.0,CAPTACAO PRECOCE PELO CANTO DA CRIANCA,2020-12-07,NaN,NaN,NaN,NaN,...,30490,5319596,42180,114.0,5.0,1.0,7.0,NaN,True,NaN


In [ ]:
df.shape

(217993, 238)

In [ ]:
for col in df.columns:
  print(col)

mc_cri_aleitamento
mc_cri_analise_conclusao
mc_cri_apgar_1_minuto
mc_cri_apgar_5_minuto
mc_cri_comentario_encaminhamento
mc_cri_data_certidao_nascimento
mc_cri_data_obito
mc_cri_data_teste_orelhinha
mc_cri_data_teste_pezinho
mc_cri_data_triagem
mc_cri_estatura
mc_cri_fator_rh
mc_cri_grupo_sanguineo
mc_cri_hora_nascimento
mc_cri_hospitalizacao
mc_cri_idade_gestacional_capurro
mc_cri_motivo_hospitalizacao
mc_cri_nome_pai
mc_cri_obito
mc_cri_obito_investigado
mc_cri_perimetro_cefalico
mc_cri_peso_nascimento
mc_cri_reanimacao
mc_cri_registrada
mc_cri_reg_maternidade
mc_cri_resultado_teste_orelhinha
mc_cri_resultado_teste_pezinho
mc_cri_resultado_triagem
mc_cri_status
mc_cri_teste_orelhinha
mc_cri_teste_pezinho
mc_cri_alimentacao
mc_cri_triagem_neo_natal
mc_cri_vacina_bcg
mc_cri_vacina_hep_b
mc_cri_vdrl
mc_pes_id_x
mc_cri_causa_obito_id
mc_par_id
mc_cri_populacao_tradicional_id
mc_acc_atual_id
mc_ult_acc_tipo_alimentacao_id
mc_mun_id_moradia_x
ativo_x_x
mc_cri_cpf
mc_cri_nis
mc_cri_data_con

---
Raw database preprocessing
---

---

Deleting lines with unpopulated VDRL

In [ ]:
df['mc_cri_vdrl'].value_counts(dropna=False)

NaN    170313
1.0     46657
0.0      1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
df.dropna(subset=['mc_cri_vdrl'], inplace=True)

In [ ]:
df['mc_cri_vdrl'].value_counts(dropna=False) # verification

1.0    46657
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
df.shape

(47680, 238)

Manual attribute selection
Here I select the attributes selected by PMCP staff

In [ ]:
columns_to_keep = [
  'mc_cri_vdrl', # mc_cri_crianca
  'mc_get_risco_gestacional', # mc_get_gestacao
  'mc_get_dt_incl', # mc_get_gestacao (it'll used to calculate 'idade')
  'mc_mul_data_nascimento', # mc_mul_mulher (it'll used to calculate 'idade')
  'mc_mul_est_civil', # mc_mul_mulher
  'mc_mul_nivel_inseguranca', # mc_mul_mulher
  'mc_mul_qtd_aborto', # mc_mul_mulher
  'mc_mul_qtd_filhos_vivos', # mc_mul_mulher
  'mc_mul_qtd_gest', # mc_mul_mulher
  'mc_mul_rec_inf_plan_fam', # mc_mul_mulher
  'mc_mul_tipo_const_casa', # mc_mul_mulher
  'mc_mul_renda_familiar', # mc_mul_mulher
  'mc_dae_escolaridade', # mc_dae_dado_evolutivo
  'mc_dae_rfa', # mc_dae_dado_evolutivo
  #'mc_pes_raca_etnia', # RECORTE
  'mc_dae_dt_incl', # mc_dae_dt_incl <- to generate wage value
  'mc_dae_dt_ult_alt', # mc_dae_dt_ult_alt <- to generate wage value
  #'mc_par_data', # RECORTE
  #'mc_mun_codigo_ibge', # RECORTE NAO USO MAIS
]

df = df[columns_to_keep]
print(f'Unified dataframe only with columns selected manually: {df.shape}')

Unified dataframe only with columns selected manually: (47680, 16)


In [ ]:
for col in df.columns:
  print(col)

mc_cri_vdrl
mc_get_risco_gestacional
mc_get_dt_incl
mc_mul_data_nascimento
mc_mul_est_civil
mc_mul_nivel_inseguranca
mc_mul_qtd_aborto
mc_mul_qtd_filhos_vivos
mc_mul_qtd_gest
mc_mul_rec_inf_plan_fam
mc_mul_tipo_const_casa
mc_mul_renda_familiar
mc_dae_escolaridade
mc_dae_rfa
mc_dae_dt_incl
mc_dae_dt_ult_alt


Creation of column age

In [ ]:
# To determinate the age of the pregnant women when they were attended by the PMCP,
# we subtracted the date of registration of the pregnancy in the SIS-MC, from the date of birth.

# After that, the age feature was created in the data set, and
# the date of registration of the pregnancy and date of birth were removed from it.

# mc_get_dt_incl = Date of pregnancy registration in the SIS-MC
# mc_mul_data_nascimento = Date of birth of the pregnant woman

test = [
         [1, 2], [3, 4], [np.nan, 0]
]

df_test = pd.DataFrame(data=test, columns=['col 1', 'col 2'])

df_test['col 3'] = df_test['col 1'] - df_test['col 2']
df_test['col 4'] = ''

for i in range(0, (df_test.shape[0])):
  if (np.isnan(df_test['col 1'].iloc[i]) == False) and (np.isnan(df_test['col 2'].iloc[i]) == False):
    df_test['col 4'].iloc[i] = df_test['col 1'].iloc[i] + df_test['col 2'].iloc[i]

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# mc_get_dt_incl = Date of pregnancy registration in the SIS-MC
# mc_mul_data_nascimento = Date of birth of the pregnant woman

print('Vazios mc_get_dt_incl', df['mc_get_dt_incl'].isna().sum())
print('Vazios mc_mul_data_nascimento', df['mc_mul_data_nascimento'].isna().sum())

Vazios mc_get_dt_incl 0
Vazios mc_mul_data_nascimento 25679


In [ ]:
df['idade'] = 0

<ipython-input-165-5efd234f7282>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['idade'] = 0


In [ ]:
df['idade']

5         0
6         0
7         0
9         0
12        0
         ..
217986    0
217987    0
217988    0
217990    0
217992    0
Name: idade, Length: 47680, dtype: int64

In [ ]:
df['idade'].value_counts()

0    47680
Name: idade, dtype: int64

In [ ]:
# mc_get_dt_incl = Date of pregnancy registration in the SIS-MC
# mc_mul_data_nascimento = Date of birth of the pregnant woman

df['mc_get_dt_incl'] # Date of pregnancy registration in the SIS-MC

5         2020-01-23 11:17:52.248
6         2019-10-29 10:03:41.846
7         2019-06-18 12:52:00.483
9         2019-09-30 14:47:11.660
12        2013-09-09 17:13:58.115
                   ...           
217986    2022-04-22 11:32:44.543
217987    2022-04-01 11:31:50.668
217988    2022-03-05 11:55:34.871
217990    2022-06-14 10:57:15.503
217992    2022-02-14 10:48:14.800
Name: mc_get_dt_incl, Length: 47680, dtype: object

In [ ]:
df['mc_mul_data_nascimento'] # Date of birth of the pregnant woman

5                NaN
6                NaN
7         1995-01-08
9         1991-01-05
12        1986-09-19
             ...    
217986           NaN
217987           NaN
217988           NaN
217990           NaN
217992           NaN
Name: mc_mul_data_nascimento, Length: 47680, dtype: object

In [ ]:
# creation of new column containing only the woman year birth
df['mc_mul_ano_nascimento'] = df['mc_mul_data_nascimento'].str[:4] # getting only the first 4 characteres

In [ ]:
df['mc_mul_ano_nascimento']

5          NaN
6          NaN
7         1995
9         1991
12        1986
          ... 
217986     NaN
217987     NaN
217988     NaN
217990     NaN
217992     NaN
Name: mc_mul_ano_nascimento, Length: 47680, dtype: object

In [ ]:
df['mc_mul_ano_nascimento'].value_counts()

1995    1356
1996    1332
1993    1313
1994    1305
1992    1261
1991    1231
1989    1219
1990    1165
1988    1138
1997    1108
1987    1022
1986     908
1985     897
1998     885
1984     728
1983     668
1982     667
1999     640
1981     521
1980     485
2000     362
1979     349
1978     279
1977     221
1976     207
1975     162
2001     142
1974     102
1973      84
1972      62
1971      40
2002      31
2015      29
1970      19
1969      18
2016      11
1968       9
2003       4
1966       3
1964       3
1967       3
2009       2
2198       2
2013       1
1940       1
2014       1
1497       1
1885       1
1197       1
2011       1
9199       1
Name: mc_mul_ano_nascimento, dtype: int64

In [ ]:
df['mc_mul_ano_nascimento'] = pd.to_numeric(df['mc_mul_ano_nascimento']) # convert to float

In [ ]:
df = df.drop(df[df['mc_mul_ano_nascimento'] < 1960].index) # drop woman with year of birth less than 1960

In [ ]:
df = df.drop(df[df['mc_mul_ano_nascimento'] > 2020].index) # drop woman with year of birth bigger than 2020

In [ ]:
df['mc_mul_ano_nascimento'].value_counts(dropna=False)

NaN       25679
1995.0     1356
1996.0     1332
1993.0     1313
1994.0     1305
1992.0     1261
1991.0     1231
1989.0     1219
1990.0     1165
1988.0     1138
1997.0     1108
1987.0     1022
1986.0      908
1985.0      897
1998.0      885
1984.0      728
1983.0      668
1982.0      667
1999.0      640
1981.0      521
1980.0      485
2000.0      362
1979.0      349
1978.0      279
1977.0      221
1976.0      207
1975.0      162
2001.0      142
1974.0      102
1973.0       84
1972.0       62
1971.0       40
2002.0       31
2015.0       29
1970.0       19
1969.0       18
2016.0       11
1968.0        9
2003.0        4
1966.0        3
1964.0        3
1967.0        3
2009.0        2
2013.0        1
2014.0        1
2011.0        1
Name: mc_mul_ano_nascimento, dtype: int64

In [ ]:
print('Vazios mc_mul_ano_nascimento', df['mc_mul_ano_nascimento'].isna().sum())

Vazios mc_mul_ano_nascimento 25679


In [ ]:
TAM_BASE = df.shape[0]
count_pos = 0
count_neg = 0

for i in range(0, TAM_BASE):
  if np.isnan(df['mc_mul_ano_nascimento'].iloc[i]) == True:
    if df['mc_cri_vdrl'].iloc[i] == 0.0:
      count_pos+=1
    else:
      count_neg+=1

print('Empty on birth date:')
print('VDRL positive', count_pos)
print('VDRL negative', count_neg)

Vazios na data de nascimento:
VDRL positivo 575
VDRL negativo 25104


In [ ]:
df.drop('mc_mul_data_nascimento', inplace=True, axis=1) # drop old birth column

In [ ]:
# mc_get_dt_incl = Date of pregnancy registration in the SIS-MC
# create new col having only the year of pregnancy registration of the woman
df['mc_get_ano_incl'] = df['mc_get_dt_incl'].str[:4] # get only the first four characteres
df['mc_get_ano_incl']

5         2020
6         2019
7         2019
9         2019
12        2013
          ... 
217986    2022
217987    2022
217988    2022
217990    2022
217992    2022
Name: mc_get_ano_incl, Length: 47673, dtype: object

In [ ]:
df['mc_get_ano_incl'] = pd.to_numeric(df['mc_get_ano_incl']) # convert to float

In [ ]:
print('Vazios mc_get_ano_incl', df['mc_get_ano_incl'].isna().sum())

Vazios mc_get_ano_incl 0


In [ ]:
df.drop('mc_get_dt_incl', inplace=True, axis=1) # drop old birth column

In [ ]:
df['mc_get_ano_incl'].value_counts(dropna=False)

2019    6125
2017    5742
2018    5318
2016    5125
2015    5002
2020    4464
2021    3890
2014    3635
2013    3066
2012    2928
2022    2378
Name: mc_get_ano_incl, dtype: int64

In [ ]:
for i in range(0, (df.shape[0])): # see if the woman birth date is bigger than the date of the gestation registration
  if (np.isnan(df['mc_mul_ano_nascimento'].iloc[i]) == False): 
    if ( (df['mc_get_ano_incl'].iloc[i]) < (df['mc_mul_ano_nascimento'].iloc[i]) ): 
      print(df['mc_get_ano_incl'].iloc[i], df['mc_mul_ano_nascimento'].iloc[i])

2012 2015.0


In [ ]:
# create a new column called age, taking the difference (gestation inclusion date - woman year of birth)
for i in range(0, (df.shape[0])): 
  if (np.isnan(df['mc_mul_ano_nascimento'].iloc[i]) == False): # the birth year must be filled
    if (df['mc_get_ano_incl'].iloc[i]) == (df['mc_mul_ano_nascimento'].iloc[i]):
      print('gestation and birth year in the same year')
    df['idade'].iloc[i] = df['mc_get_ano_incl'].iloc[i] - df['mc_mul_ano_nascimento'].iloc[i]

gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nascimento no mesmo ano
gestaçao e nasciment

In [ ]:
# the ages that are not filled are considered missing
df['idade'].value_counts() 

 0     25710
 22     1277
 24     1274
 20     1259
 23     1248
 25     1243
 27     1196
 21     1195
 19     1164
 26     1161
 28     1073
 18     1015
 29      986
 30      963
 31      818
 17      789
 32      750
 33      632
 16      630
 34      595
 35      447
 36      396
 15      352
 37      320
 38      249
 39      233
 40      190
 14      145
 41      110
 42       78
 43       48
 44       35
 13       31
 45       23
 46        7
 48        6
 47        5
 5         3
 3         2
 1         2
 2         2
 7         2
 12        2
 4         1
 6         1
 11        1
-3         1
 50        1
 49        1
 51        1
Name: idade, dtype: int64

In [ ]:
df.shape

(47673, 17)

In [ ]:
df['mc_cri_vdrl'].value_counts()

1.0    46650
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
# remove age that are outliers
TAM_BASE = df.shape[0]

for i in range(0, TAM_BASE): 
  if (df['idade'].iloc[i] < 11) and (df['idade'].iloc[i] != 0):
    print(df['idade'].iloc[i]) # the index is relloced after removal, so navegating through the dataframe will not work

6
3
3
5
4
1
2
7
-3
1
2
7
5
5


In [ ]:
TAM_BASE = df.shape[0]
age_to_delete = []
idx_age_to_delete = []

for i in range(0, TAM_BASE):
  if (df['idade'].iloc[i] < 11) and (df['idade'].iloc[i] != 0):
    age_to_delete.append(df['idade'].iloc[i])
    idx_age_to_delete.append(i)

In [ ]:
print('idade')
for i in age_to_delete:
  print(i)

print('idx')
for i in idx_age_to_delete:
  print(i)

idade
6
3
3
5
4
1
2
7
-3
1
2
7
5
5
idx
246
1047
2353
2805
4591
8547
8585
8911
12480
14941
17810
29180
31161
32471


In [ ]:
df.shape

(47673, 17)

In [ ]:
df = df[ ~(df['idade'].isin(age_to_delete)) ]

In [ ]:
df.shape

(47659, 17)

In [ ]:
df['mc_cri_vdrl'].value_counts(dropna=False)

1.0    46636
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
df = df.drop(columns=['mc_mul_ano_nascimento'], axis=1)

In [ ]:
df['idade'].value_counts(dropna=False) # check if everything is alright

0     25710
22     1277
24     1274
20     1259
23     1248
25     1243
27     1196
21     1195
19     1164
26     1161
28     1073
18     1015
29      986
30      963
31      818
17      789
32      750
33      632
16      630
34      595
35      447
36      396
15      352
37      320
38      249
39      233
40      190
14      145
41      110
42       78
43       48
44       35
13       31
45       23
46        7
48        6
47        5
12        2
11        1
50        1
49        1
51        1
Name: idade, dtype: int64

Other outliers exclusion

In [ ]:
df = df.dropna(subset=['mc_mul_renda_familiar'])
print(df.shape)

(47648, 16)


In [ ]:
df['mc_dae_rfa'].value_counts(dropna=False)

NaN         12950
400.00       1878
200.00       1821
300.00       1712
500.00       1506
            ...  
511.11          1
634.00          1
1115.00         1
779.00          1
48800.00        1
Name: mc_dae_rfa, Length: 1296, dtype: int64

In [ ]:
TAM_BASE = df.shape[0]

for i in range(0, TAM_BASE):
  if df['mc_dae_rfa'].iloc[i] > 20000:
    print('hi')

oi
oi
oi
oi
oi
oi
oi
oi
oi
oi


In [ ]:
df = df[~(df['mc_dae_rfa'] > 20000)] # remove family wage bigger than 20.000

In [ ]:
df['mc_dae_rfa'].value_counts(dropna=False)

NaN         12950
400.00       1878
200.00       1821
300.00       1712
500.00       1506
            ...  
14200.00        1
229.24          1
3.60            1
754.00          1
2130.00         1
Name: mc_dae_rfa, Length: 1286, dtype: int64

In [ ]:
df['mc_cri_vdrl'].value_counts()

1.0    46615
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
# check other outliers
for col in df.columns:
  print(col)

mc_cri_vdrl
mc_get_risco_gestacional
mc_mul_est_civil
mc_mul_nivel_inseguranca
mc_mul_qtd_aborto
mc_mul_qtd_filhos_vivos
mc_mul_qtd_gest
mc_mul_rec_inf_plan_fam
mc_mul_tipo_const_casa
mc_mul_renda_familiar
mc_dae_escolaridade
mc_dae_rfa
mc_dae_dt_incl
mc_dae_dt_ult_alt
idade
mc_get_ano_incl


In [ ]:
df['mc_mul_qtd_aborto'].value_counts(dropna=False)

NaN     24842
0.0     15505
1.0      5770
2.0      1146
3.0       248
4.0        51
11.0       27
5.0        19
6.0         8
22.0        5
7.0         3
10.0        3
8.0         2
12.0        2
20.0        1
21.0        1
13.0        1
9.0         1
30.0        1
14.0        1
33.0        1
Name: mc_mul_qtd_aborto, dtype: int64

In [ ]:
df['mc_mul_qtd_filhos_vivos'].value_counts(dropna=False)

NaN     21359
1.0     11243
2.0      5664
0.0      5206
3.0      2299
4.0       960
5.0       388
6.0       233
7.0       136
8.0        76
9.0        32
10.0       23
12.0        7
11.0        4
14.0        2
21.0        1
44.0        1
17.0        1
36.0        1
15.0        1
22.0        1
Name: mc_mul_qtd_filhos_vivos, dtype: int64

In [ ]:
df.shape

(47638, 16)

In [ ]:
df_pos = df[df['mc_cri_vdrl'] == 0.0]
df_neg = df[df['mc_cri_vdrl'] == 1.0]

In [ ]:
df_pos.shape

(1023, 16)

In [ ]:
df_pos['mc_mul_qtd_filhos_vivos'].value_counts(dropna=False)

NaN     409
1.0     226
2.0     154
0.0     122
3.0      58
4.0      27
6.0      11
5.0       8
7.0       6
12.0      1
9.0       1
Name: mc_mul_qtd_filhos_vivos, dtype: int64

In [ ]:
df_neg.shape

(46615, 16)

In [ ]:
df_neg['mc_mul_qtd_filhos_vivos'].value_counts(dropna=False)

NaN     20950
1.0     11017
2.0      5510
0.0      5084
3.0      2241
4.0       933
5.0       380
6.0       222
7.0       130
8.0        76
9.0        31
10.0       23
12.0        6
11.0        4
14.0        2
21.0        1
44.0        1
17.0        1
36.0        1
15.0        1
22.0        1
Name: mc_mul_qtd_filhos_vivos, dtype: int64

In [ ]:
df = df[~(df['mc_mul_qtd_filhos_vivos'] > 15)]

In [ ]:
df['mc_cri_vdrl'].value_counts()

1.0    46610
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
df.shape

(47633, 16)

In [ ]:
df['mc_mul_qtd_filhos_vivos'].value_counts(dropna=False)

NaN     21359
1.0     11243
2.0      5664
0.0      5206
3.0      2299
4.0       960
5.0       388
6.0       233
7.0       136
8.0        76
9.0        32
10.0       23
12.0        7
11.0        4
14.0        2
15.0        1
Name: mc_mul_qtd_filhos_vivos, dtype: int64

In [ ]:
df_pos['mc_mul_qtd_aborto'].value_counts() # if positive

0.0     328
1.0     164
2.0      28
3.0       6
4.0       1
6.0       1
22.0      1
11.0      1
Name: mc_mul_qtd_aborto, dtype: int64

In [ ]:
df['mc_mul_qtd_aborto'].replace(22, np.nan, inplace=True) # to not lose data

In [ ]:
df['mc_mul_qtd_aborto'].value_counts()

0.0     15504
1.0      5769
2.0      1145
3.0       248
4.0        51
11.0       27
5.0        19
6.0         8
7.0         3
10.0        3
8.0         2
12.0        2
20.0        1
21.0        1
13.0        1
9.0         1
30.0        1
14.0        1
33.0        1
Name: mc_mul_qtd_aborto, dtype: int64

In [ ]:
df = df[~(df['mc_mul_qtd_aborto'] > 15)] 

In [ ]:
df['mc_cri_vdrl'].value_counts(dropna=False)

1.0    46606
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
df['mc_mul_qtd_gest'].value_counts(dropna=False)

1.0     13271
2.0      9443
NaN      9350
3.0      5168
0.0      4912
4.0      2592
5.0      1258
6.0       619
7.0       315
8.0       200
9.0       104
11.0      102
10.0       67
12.0       39
22.0       38
33.0       19
21.0       17
13.0       16
20.0       16
14.0       15
31.0       10
15.0        7
44.0        7
40.0        5
23.0        5
32.0        5
54.0        4
77.0        3
16.0        3
53.0        2
34.0        2
66.0        2
55.0        2
64.0        2
30.0        2
52.0        1
19.0        1
71.0        1
17.0        1
42.0        1
96.0        1
72.0        1
Name: mc_mul_qtd_gest, dtype: int64

In [ ]:
df_pos['mc_mul_qtd_gest'].value_counts(dropna=False)

1.0     250
2.0     214
NaN     195
3.0     126
0.0      99
4.0      53
5.0      41
6.0      16
10.0      7
8.0       6
7.0       6
9.0       5
11.0      2
22.0      1
21.0      1
16.0      1
Name: mc_mul_qtd_gest, dtype: int64

In [ ]:
df['mc_mul_qtd_gest'].replace(16, np.nan, inplace=True) # to not lose data
df['mc_mul_qtd_gest'].replace(21, np.nan, inplace=True) # to not lose data
df['mc_mul_qtd_gest'].replace(22, np.nan, inplace=True) # to not lose data

In [ ]:
TAM_BASE = df.shape[0]

for i in range(0, TAM_BASE):
  if df['mc_mul_qtd_filhos_vivos'].iloc[i] > df['mc_mul_qtd_gest'].iloc[i]:
    print(df['mc_mul_qtd_filhos_vivos'].iloc[i], df['mc_mul_qtd_gest'].iloc[i])

3.0 2.0
3.0 2.0
8.0 5.0
10.0 9.0
10.0 9.0
2.0 1.0
1.0 0.0
1.0 0.0
3.0 2.0
3.0 2.0
5.0 4.0
1.0 0.0
1.0 0.0
3.0 2.0
2.0 1.0
2.0 1.0
2.0 1.0
2.0 1.0
3.0 2.0
2.0 1.0
2.0 1.0
9.0 8.0
4.0 3.0
2.0 1.0
6.0 5.0
2.0 1.0
2.0 1.0
4.0 3.0
11.0 1.0
4.0 2.0
4.0 2.0
2.0 1.0
1.0 0.0
3.0 2.0
1.0 0.0
2.0 1.0
3.0 2.0
3.0 2.0
2.0 1.0
1.0 0.0
2.0 0.0
2.0 1.0
2.0 1.0
5.0 4.0
6.0 5.0
9.0 8.0
9.0 8.0
2.0 1.0
9.0 8.0
1.0 0.0
8.0 2.0
4.0 3.0
4.0 3.0
1.0 0.0
1.0 0.0
5.0 4.0
1.0 0.0
2.0 1.0
2.0 1.0
12.0 2.0
12.0 2.0
1.0 0.0
1.0 0.0
1.0 0.0
2.0 1.0
1.0 0.0
1.0 0.0
2.0 1.0
2.0 1.0
4.0 3.0
3.0 2.0
3.0 2.0
4.0 0.0
7.0 6.0
4.0 2.0
4.0 2.0
1.0 0.0
2.0 1.0
2.0 1.0
6.0 5.0
2.0 1.0
3.0 2.0
2.0 1.0
2.0 1.0
3.0 2.0
2.0 0.0
4.0 1.0
7.0 2.0
7.0 2.0
1.0 0.0
2.0 1.0
3.0 2.0
3.0 2.0
2.0 1.0
2.0 1.0
2.0 1.0
2.0 1.0
12.0 2.0
4.0 1.0
2.0 0.0
11.0 1.0
2.0 1.0
2.0 1.0
2.0 1.0
1.0 0.0
2.0 1.0
2.0 1.0
6.0 5.0
3.0 2.0
2.0 1.0
2.0 1.0
2.0 1.0
2.0 1.0
4.0 3.0
1.0 0.0
1.0 0.0
1.0 0.0
2.0 1.0
2.0 1.0
2.0 1.0
2.0 1.0
15.0 14.0
1.0 0.0
3.0 2.0

In [ ]:
df_pos['mc_mul_qtd_aborto'].value_counts(dropna=False)

NaN     493
0.0     328
1.0     164
2.0      28
3.0       6
4.0       1
6.0       1
22.0      1
11.0      1
Name: mc_mul_qtd_aborto, dtype: int64

In [ ]:
df_neg['mc_mul_qtd_aborto'].value_counts(dropna=False)

NaN     24349
0.0     15177
1.0      5606
2.0      1118
3.0       242
4.0        50
11.0       26
5.0        19
6.0         7
22.0        4
7.0         3
10.0        3
8.0         2
12.0        2
20.0        1
21.0        1
13.0        1
9.0         1
30.0        1
14.0        1
33.0        1
Name: mc_mul_qtd_aborto, dtype: int64

In [ ]:
df_pos['mc_mul_qtd_gest'].value_counts(dropna=False)

1.0     250
2.0     214
NaN     195
3.0     126
0.0      99
4.0      53
5.0      41
6.0      16
10.0      7
8.0       6
7.0       6
9.0       5
11.0      2
22.0      1
21.0      1
16.0      1
Name: mc_mul_qtd_gest, dtype: int64

In [ ]:
df_neg['mc_mul_qtd_gest'].value_counts(dropna=False)

1.0     13022
2.0      9229
NaN      9155
3.0      5044
0.0      4813
4.0      2541
5.0      1218
6.0       603
7.0       309
8.0       194
11.0      100
9.0        99
10.0       60
12.0       39
22.0       37
33.0       19
20.0       17
13.0       16
21.0       16
14.0       15
31.0       10
15.0        7
44.0        7
23.0        5
40.0        5
32.0        5
54.0        4
66.0        3
77.0        3
64.0        2
55.0        2
34.0        2
53.0        2
30.0        2
16.0        2
42.0        1
71.0        1
96.0        1
72.0        1
52.0        1
19.0        1
17.0        1
99.0        1
Name: mc_mul_qtd_gest, dtype: int64

In [ ]:
df = df[~(df['mc_mul_qtd_gest'] > 15)] 

In [ ]:
df['mc_cri_vdrl'].value_counts()

1.0    46513
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
for col in df.columns: # analysing cols
  print(col)

mc_cri_vdrl
mc_get_risco_gestacional
mc_mul_est_civil
mc_mul_nivel_inseguranca
mc_mul_qtd_aborto
mc_mul_qtd_filhos_vivos
mc_mul_qtd_gest
mc_mul_rec_inf_plan_fam
mc_mul_tipo_const_casa
mc_mul_renda_familiar
mc_dae_escolaridade
mc_dae_rfa
mc_dae_dt_incl
mc_dae_dt_ult_alt
idade
mc_get_ano_incl


In [ ]:
df['mc_cri_vdrl'].value_counts(dropna=False)

1.0    46513
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
df['mc_dae_rfa'].value_counts(dropna=False) # preprocess, check the data distribution

NaN         12923
400.00       1874
200.00       1820
300.00       1708
500.00       1505
            ...  
800.80          1
14200.00        1
229.24          1
3.60            1
2130.00         1
Name: mc_dae_rfa, Length: 1285, dtype: int64

In [ ]:
df['idade'].value_counts(dropna=False) # put 0 as nan

0     25657
22     1275
24     1269
20     1255
23     1242
25     1241
21     1190
27     1185
19     1163
26     1156
28     1069
18     1013
29      983
30      960
31      814
17      788
32      749
33      630
16      630
34      594
35      447
36      396
15      352
37      318
38      247
39      232
40      190
14      145
41      109
42       78
43       46
44       35
13       31
45       23
46        7
48        6
47        5
12        2
11        1
50        1
49        1
51        1
Name: idade, dtype: int64

In [ ]:
df['idade'].replace(0, np.nan, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
df['idade'].value_counts(dropna=False)

NaN     25657
22.0     1275
24.0     1269
20.0     1255
23.0     1242
25.0     1241
21.0     1190
27.0     1185
19.0     1163
26.0     1156
28.0     1069
18.0     1013
29.0      983
30.0      960
31.0      814
17.0      788
32.0      749
33.0      630
16.0      630
34.0      594
35.0      447
36.0      396
15.0      352
37.0      318
38.0      247
39.0      232
40.0      190
14.0      145
41.0      109
42.0       78
43.0       46
44.0       35
13.0       31
45.0       23
46.0        7
48.0        6
47.0        5
12.0        2
11.0        1
50.0        1
49.0        1
51.0        1
Name: idade, dtype: int64

In [ ]:
df['mc_mul_qtd_aborto'].value_counts(dropna=False)

NaN     24808
0.0     15467
1.0      5761
2.0      1143
3.0       246
4.0        50
11.0       23
5.0        19
6.0         8
7.0         3
10.0        3
8.0         2
14.0        1
12.0        1
9.0         1
Name: mc_mul_qtd_aborto, dtype: int64

In [ ]:
df['mc_mul_qtd_filhos_vivos'].value_counts(dropna=False)

NaN     21345
1.0     11220
2.0      5630
0.0      5203
3.0      2287
4.0       956
5.0       384
6.0       232
7.0       135
8.0        75
9.0        32
10.0       23
12.0        7
11.0        4
14.0        2
15.0        1
Name: mc_mul_qtd_filhos_vivos, dtype: int64

Exclude values of mc_dae_dt_ult_alt that do not have the last alteration date

In [ ]:
df['mc_dae_dt_ult_alt'].value_counts(dropna=False)

NaN                        20
2022-07-07 11:46:08.535     4
2022-08-26 08:52:01.247     4
2022-09-11 09:35:49.882     4
2022-04-12 11:50:36.697     4
                           ..
2018-07-10 21:35:52.209     1
2019-04-09 10:01:00.601     1
2017-07-20 23:43:04.781     1
2016-01-19 09:50:33.673     1
2022-09-30 15:20:38.762     1
Name: mc_dae_dt_ult_alt, Length: 43250, dtype: int64

In [ ]:
df['mc_dae_dt_incl'].value_counts(dropna=False)

2012-12-21 00:00:00.000    20
2022-07-07 11:46:08.535     4
2022-08-26 08:52:01.247     4
2022-09-11 09:35:49.882     4
2022-04-12 11:50:36.697     4
                           ..
2018-07-10 21:35:52.209     1
2019-04-09 10:01:00.601     1
2017-07-20 23:43:04.781     1
2016-01-19 09:50:33.673     1
2022-09-30 15:20:38.762     1
Name: mc_dae_dt_incl, Length: 43250, dtype: int64

In [ ]:
df['mc_dae_dt_ult_alt'] = df['mc_dae_dt_ult_alt'].str[:4]

In [ ]:
df['mc_dae_dt_ult_alt'].value_counts(dropna=False)

2022    9897
2019    7482
2020    7276
2018    5482
2021    4926
2016    4403
2017    4293
2015    2168
2014    1064
2013     525
NaN       20
Name: mc_dae_dt_ult_alt, dtype: int64

In [ ]:
TAM_BASE = df.shape[0]

for i in range(0, TAM_BASE):
  if type(df['mc_dae_dt_ult_alt'].iloc[i]) != str:
    print(df['mc_dae_rfa'].iloc[i])

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
df.dropna(subset=['mc_dae_dt_ult_alt'], inplace=True)

In [ ]:
df['mc_dae_dt_ult_alt'].value_counts(dropna=False)

2022    9897
2019    7482
2020    7276
2018    5482
2021    4926
2016    4403
2017    4293
2015    2168
2014    1064
2013     525
Name: mc_dae_dt_ult_alt, dtype: int64

In [ ]:
df['mc_dae_dt_incl'].value_counts(dropna=False)

2022-03-16 09:40:51.764    4
2022-07-07 11:46:08.535    4
2021-02-11 13:58:39.400    4
2018-11-12 09:31:14.943    4
2022-04-12 11:50:36.697    4
                          ..
2018-07-10 21:35:52.209    1
2019-04-09 10:01:00.601    1
2017-07-20 23:43:04.781    1
2016-01-19 09:50:33.673    1
2022-09-30 15:20:38.762    1
Name: mc_dae_dt_incl, Length: 43249, dtype: int64

In [ ]:
TAM_BASE = df.shape[0]

for i in range(0, TAM_BASE):
  if type(df['mc_dae_dt_ult_alt'].iloc[i]) != str:
    print(df['mc_cri_vdrl'].iloc[i])

In [ ]:
df['mc_dae_dt_incl'] = df['mc_dae_dt_incl'].str[:4]

In [ ]:
df['mc_dae_dt_incl'].value_counts(dropna=False)

2022    9897
2019    7481
2020    7277
2018    5484
2021    4924
2016    4403
2017    4293
2015    2168
2014    1064
2013     525
Name: mc_dae_dt_incl, dtype: int64

In [ ]:
df.shape

(47516, 16)

In [ ]:
# version 2
df['mc_dae_dt_ult_alt'].value_counts()

2022    9897
2019    7482
2020    7276
2018    5482
2021    4926
2016    4403
2017    4293
2015    2168
2014    1064
2013     525
Name: mc_dae_dt_ult_alt, dtype: int64

Column to ajust the wages according to the year

In [ ]:
df['mc_dae_valor_salario_por_ano'] = None

In [ ]:
df['mc_dae_valor_salario_por_ano'].value_counts(dropna=False)

NaN    47516
Name: mc_dae_valor_salario_por_ano, dtype: int64

In [ ]:
df['mc_dae_dt_ult_alt'] = pd.to_numeric(df['mc_dae_dt_ult_alt']) # convert

In [ ]:
type(df['mc_dae_dt_ult_alt'].iloc[122])

numpy.int64

Adjust the wages, according to the year

In [ ]:
def return_corrected_value(value, ano): # receives the wage value
  if ano == 2013: # 678 reais
    sal_min = 678
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2014: # 724 reais
    sal_min = 724
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2015: # 788 reais
    sal_min = 788
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2016: # 880 reais
    sal_min = 880
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2017: # 937 reais
    sal_min = 937
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2018: # 954 reais
    sal_min = 954
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2019: # 998 reais
    sal_min = 998
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2020: # 1039 reais
    sal_min = 1039
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2021: # 1100 reais
    sal_min = 1100
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  elif ano == 2022: # 1212 reais
    sal_min = 1212
    if (value < (sal_min/4) and (value >= 0)) or value == 0.0:
      return 0
    if (value >= (sal_min/4)) and (value < (sal_min/2)):
      return 1
    if (value >= (sal_min/2)) and (value < (sal_min)):
      return 2
    if (value >= (sal_min)) and (value < (2*sal_min)):
      return 3
    if (value >= (2*sal_min)):
      return 4
  else:
    return None

In [ ]:
def alter_wage_by_year(df):
  TAM_BASE = df.shape[0]
  for i in range(0, TAM_BASE):
    if ((df['mc_dae_valor_salario_por_ano'].iloc[i]) == None) and (np.isnan(df['mc_dae_rfa'].iloc[i]) == False):
      value = return_corrected_value(df['mc_dae_rfa'].iloc[i], df['mc_dae_dt_ult_alt'].iloc[i])
      if value != None:
        #print(value)
        df['mc_dae_rfa'].iloc[i] = value

In [ ]:
alter_wage_by_year(df)

In [ ]:
df['mc_dae_rfa'].value_counts(dropna=False)

NaN    12903
2.0    11607
0.0    10547
1.0     7575
3.0     4065
4.0      819
Name: mc_dae_rfa, dtype: int64

In [ ]:
df = df.drop(columns=['mc_dae_dt_incl', 'mc_dae_dt_ult_alt'], axis=1) # drop this column because it will not be used anymore

In [ ]:
df = df.drop(columns=['mc_get_ano_incl', 'mc_dae_valor_salario_por_ano'], axis=1) #  drop this column because it will not be used anymore

In [ ]:
df.shape

(47516, 13)

In [ ]:
df['mc_mul_renda_familiar'].value_counts(dropna=False)

0.0    35567
1.0    11949
Name: mc_mul_renda_familiar, dtype: int64

In [ ]:
df['mc_dae_rfa'].value_counts(dropna=False)

NaN    12903
2.0    11607
0.0    10547
1.0     7575
3.0     4065
4.0      819
Name: mc_dae_rfa, dtype: int64

In [ ]:
TAM_BASE = df.shape[0]
count_nan = 0
count_zero = 0
count_others = 0

for i in range(0, TAM_BASE):
  if df['mc_mul_renda_familiar'].iloc[i] == 1.0:
    if np.isnan(df['mc_dae_rfa'].iloc[i]) == True:
      count_nan+=1
    elif (df['mc_dae_rfa'].iloc[i]) == 0.0:
      count_zero+=1
    else:
      #print(df['mc_dae_rfa'].iloc[i])
      count_others+=1

print('Data of wage empty:', count_nan)
print('Data of wage as zero:', count_zero)
print('Data of wage filled:', count_others)

Dados da renda vazios: 9233
Dados da renda como zero: 655
Dados da renda preenchidos: 2061


In [ ]:
for col in df.columns:
  print(col)

mc_cri_vdrl
mc_get_risco_gestacional
mc_mul_est_civil
mc_mul_nivel_inseguranca
mc_mul_qtd_aborto
mc_mul_qtd_filhos_vivos
mc_mul_qtd_gest
mc_mul_rec_inf_plan_fam
mc_mul_tipo_const_casa
mc_mul_renda_familiar
mc_dae_escolaridade
mc_dae_rfa
idade


In [ ]:
df['mc_cri_vdrl'].value_counts()

1.0    46493
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
df.drop(columns=['mc_mul_renda_familiar'], axis=1, inplace=True)

In [ ]:
df.shape

(47516, 12)

In [ ]:
for col in df.columns:
  print(df[col].value_counts(dropna=False))

1.0    46493
0.0     1023
Name: mc_cri_vdrl, dtype: int64
1.0    38883
0.0     6257
NaN     1428
2.0      948
Name: mc_get_risco_gestacional, dtype: int64
5.0    18053
0.0    15578
1.0    11401
NaN     1971
4.0      319
3.0      111
2.0       83
Name: mc_mul_est_civil, dtype: int64
NaN    20756
0.0    19003
1.0     4592
2.0     1211
4.0     1178
3.0      402
5.0      340
6.0       34
Name: mc_mul_nivel_inseguranca, dtype: int64
NaN     24795
0.0     15463
1.0      5758
2.0      1143
3.0       246
4.0        50
11.0       23
5.0        19
6.0         8
7.0         3
10.0        3
8.0         2
14.0        1
12.0        1
9.0         1
Name: mc_mul_qtd_aborto, dtype: int64
NaN     21325
1.0     11220
2.0      5630
0.0      5203
3.0      2287
4.0       956
5.0       384
6.0       232
7.0       135
8.0        75
9.0        32
10.0       23
12.0        7
11.0        4
14.0        2
15.0        1
Name: mc_mul_qtd_filhos_vivos, dtype: int64
1.0     13264
2.0      9443
NaN      9398
3.0      5

In [ ]:
df['mc_cri_vdrl'].value_counts()

1.0    46493
0.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
test = df['mc_mul_nivel_inseguranca'].value_counts()
print(len(test))

7


In [ ]:
df['mc_mul_nivel_inseguranca'].value_counts()

0.0    19003
1.0     4592
2.0     1211
4.0     1178
3.0      402
5.0      340
6.0       34
Name: mc_mul_nivel_inseguranca, dtype: int64

Inverting the values, because in the PMCP base 0 is positive and 1 is negative

In [ ]:
for col in df.columns:
  qtd = len(df[col].value_counts())
  if qtd <= 2:
    print(col)

mc_cri_vdrl
mc_mul_rec_inf_plan_fam


In [ ]:
# adjust values
df.loc[df['mc_cri_vdrl'] == 0.0, 'mc_cri_vdrl'] = 2.0
df.loc[df['mc_cri_vdrl'] == 1.0, 'mc_cri_vdrl'] = 0.0
df.loc[df['mc_cri_vdrl'] == 2.0, 'mc_cri_vdrl'] = 1.0

df['mc_cri_vdrl'].value_counts()

0.0    46493
1.0     1023
Name: mc_cri_vdrl, dtype: int64

In [ ]:
df['mc_mul_rec_inf_plan_fam'].value_counts()

0.0    22751
1.0    13805
Name: mc_mul_rec_inf_plan_fam, dtype: int64

In [ ]:
# adjust values
df.loc[df['mc_mul_rec_inf_plan_fam'] == 0.0, 'mc_mul_rec_inf_plan_fam'] = 2.0
df.loc[df['mc_mul_rec_inf_plan_fam'] == 1.0, 'mc_mul_rec_inf_plan_fam'] = 0.0
df.loc[df['mc_mul_rec_inf_plan_fam'] == 2.0, 'mc_mul_rec_inf_plan_fam'] = 1.0

df['mc_mul_rec_inf_plan_fam'].value_counts()

1.0    22751
0.0    13805
Name: mc_mul_rec_inf_plan_fam, dtype: int64

In [ ]:
df['mc_get_risco_gestacional'].value_counts(dropna=False)

1.0    38883
0.0     6257
NaN     1428
2.0      948
Name: mc_get_risco_gestacional, dtype: int64

In [ ]:
# adjust values
df.loc[df['mc_get_risco_gestacional'] == 0.0, 'mc_get_risco_gestacional'] = 3.0
df.loc[df['mc_get_risco_gestacional'] == 1.0, 'mc_get_risco_gestacional'] = 0.0
df.loc[df['mc_get_risco_gestacional'] == 3.0, 'mc_get_risco_gestacional'] = 1.0

In [ ]:
df['mc_get_risco_gestacional'].value_counts(dropna=False)

0.0    38883
1.0     6257
NaN     1428
2.0      948
Name: mc_get_risco_gestacional, dtype: int64

In [ ]:
df['mc_dae_escolaridade'].value_counts()

5.0    15382
4.0    10248
6.0     7835
2.0     5660
3.0     2835
1.0     1609
7.0     1258
8.0      959
0.0      557
Name: mc_dae_escolaridade, dtype: int64

In [ ]:
df['mc_get_risco_gestacional'].value_counts()

0.0    38883
1.0     6257
2.0      948
Name: mc_get_risco_gestacional, dtype: int64

In [ ]:
df['mc_mul_est_civil'].value_counts()

5.0    18053
0.0    15578
1.0    11401
4.0      319
3.0      111
2.0       83
Name: mc_mul_est_civil, dtype: int64

In [ ]:
df['mc_mul_nivel_inseguranca'].value_counts()

0.0    19003
1.0     4592
2.0     1211
4.0     1178
3.0      402
5.0      340
6.0       34
Name: mc_mul_nivel_inseguranca, dtype: int64

In [ ]:
df['mc_mul_rec_inf_plan_fam'].value_counts()

1.0    22751
0.0    13805
Name: mc_mul_rec_inf_plan_fam, dtype: int64

Numeric columns categorization

In [ ]:
df['idade'] = np.where(df['idade'].between(11, 19), 0, df['idade']) # age categorization
df['idade'] = np.where(df['idade'].between(20, 51), 1, df['idade'])

df['idade'].value_counts(dropna=False)

NaN    25643
1.0    17751
0.0     4122
Name: idade, dtype: int64

In [ ]:
df['mc_mul_qtd_aborto'] = np.where(df['mc_mul_qtd_aborto'].between(0, 4), 0, df['mc_mul_qtd_aborto']) # qtd abortos
df['mc_mul_qtd_aborto'] = np.where(df['mc_mul_qtd_aborto'].between(5, 9), 1, df['mc_mul_qtd_aborto'])
df['mc_mul_qtd_aborto'] = np.where(df['mc_mul_qtd_aborto'].between(10, 15), 2, df['mc_mul_qtd_aborto'])

df['mc_mul_qtd_aborto'].value_counts(dropna=False)

NaN    24795
0.0    22660
1.0       33
2.0       28
Name: mc_mul_qtd_aborto, dtype: int64

In [ ]:
df['mc_mul_qtd_gest'] = np.where(df['mc_mul_qtd_gest'].between(0, 4), 0, df['mc_mul_qtd_gest']) # qtd gestacoes
df['mc_mul_qtd_gest'] = np.where(df['mc_mul_qtd_gest'].between(5, 9), 1, df['mc_mul_qtd_gest'])
df['mc_mul_qtd_gest'] = np.where(df['mc_mul_qtd_gest'].between(10, 15), 2, df['mc_mul_qtd_gest'])

df['mc_mul_qtd_gest'].value_counts(dropna=False)

0.0    35377
NaN     9398
1.0     2496
2.0      245
Name: mc_mul_qtd_gest, dtype: int64

In [ ]:
df['mc_mul_qtd_filhos_vivos'] = np.where(df['mc_mul_qtd_filhos_vivos'].between(0, 4), 0, df['mc_mul_qtd_filhos_vivos']) # qtd filhos vivos
df['mc_mul_qtd_filhos_vivos'] = np.where(df['mc_mul_qtd_filhos_vivos'].between(5, 9), 1, df['mc_mul_qtd_filhos_vivos'])
df['mc_mul_qtd_filhos_vivos'] = np.where(df['mc_mul_qtd_filhos_vivos'].between(10, 15), 2, df['mc_mul_qtd_filhos_vivos'])

In [ ]:
for col in df.columns:
  print(df[col].value_counts())

0.0    46493
1.0     1023
Name: mc_cri_vdrl, dtype: int64
0.0    38883
1.0     6257
2.0      948
Name: mc_get_risco_gestacional, dtype: int64
5.0    18053
0.0    15578
1.0    11401
4.0      319
3.0      111
2.0       83
Name: mc_mul_est_civil, dtype: int64
0.0    19003
1.0     4592
2.0     1211
4.0     1178
3.0      402
5.0      340
6.0       34
Name: mc_mul_nivel_inseguranca, dtype: int64
0.0    22660
1.0       33
2.0       28
Name: mc_mul_qtd_aborto, dtype: int64
0.0    25296
1.0      858
2.0       37
Name: mc_mul_qtd_filhos_vivos, dtype: int64
0.0    35377
1.0     2496
2.0      245
Name: mc_mul_qtd_gest, dtype: int64
1.0    22751
0.0    13805
Name: mc_mul_rec_inf_plan_fam, dtype: int64
4.0    43344
2.0      746
5.0      559
1.0      164
0.0      159
3.0       83
Name: mc_mul_tipo_const_casa, dtype: int64
5.0    15382
4.0    10248
6.0     7835
2.0     5660
3.0     2835
1.0     1609
7.0     1258
8.0      959
0.0      557
Name: mc_dae_escolaridade, dtype: int64
2.0    11607
0.0    1054

In [ ]:
df.head()

,mc_cri_vdrl,mc_get_risco_gestacional,mc_mul_est_civil,mc_mul_nivel_inseguranca,mc_mul_qtd_aborto,mc_mul_qtd_filhos_vivos,mc_mul_qtd_gest,mc_mul_rec_inf_plan_fam,mc_mul_tipo_const_casa,mc_dae_escolaridade,mc_dae_rfa,idade
5,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,4.0,5.0,0.0,NaN
6,0.0,0.0,5.0,0.0,NaN,NaN,0.0,0.0,4.0,5.0,3.0,NaN
7,0.0,0.0,5.0,NaN,0.0,0.0,0.0,1.0,4.0,6.0,NaN,1.0
9,0.0,1.0,5.0,NaN,NaN,0.0,0.0,1.0,4.0,2.0,1.0,1.0
12,0.0,0.0,1.0,0.0,NaN,0.0,0.0,1.0,4.0,4.0,2.0,1.0


In [ ]:
df.to_csv(path+'base_pre_proc_desb_com_nan.csv', index=False) # database preprocessed, imbalanced and with nan